In [36]:
import json

# Pfad zur Datei
# file_path = "Spider-Syn-main/preprocessed_dataset/dev.json"
file_path = "Spider-Syn-main/preprocessed_dataset/tables.json"
# file_path = "Spider-Syn-main/preprocessed_dataset/train_spider.json"


# JSON-Datei laden
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Struktur analysieren (rekursiv alle Schlüssel extrahieren)
def extract_structure(obj):
    if isinstance(obj, dict):
        return {k: extract_structure(v) for k, v in obj.items()}
    elif isinstance(obj, list) and obj:
        return [extract_structure(obj[0])]
    else:
        return type(obj).__name__
    

# Struktur ausgeben
structure = extract_structure(data)
structure


[{'column_names': [['int']],
  'column_names_original': [['int']],
  'column_types': ['str'],
  'db_id': 'str',
  'foreign_keys': [['int']],
  'primary_keys': ['int'],
  'table_names': ['str'],
  'table_names_original': ['str']}]

In [62]:
from typing import List, Dict, Optional
import numpy as np
import sqlite3

class QuestionAnswerMapping:
    """
    Speichert die Zuordnung zwischen:
      - db_id: Identifikation der Datenbank
      - spider_question: Die originale Frage aus dem Spider-Datenset
      - spider_syn_question: Die synthetisierte Version der Frage (SpiderSyn)
      - query: Das zugehörige SQL-Query (aus dem Datenset)
      - generated_query: Ein von der Text-to-SQL Komponente generierter Query
      - spider_syn_embedding: Ein Embedding-Vektor für die SpiderSyn-Frage
    """
    def __init__(self, 
                 db_id: str, 
                 spider_question: str, 
                 spider_syn_question: str, 
                 query: str, 
                 generated_query: Optional[str] = None,
                 spider_syn_embedding: Optional[np.ndarray] = None):
        self.db_id = db_id
        self.spider_question = spider_question
        self.spider_syn_question = spider_syn_question
        self.query = query
        self.generated_query = generated_query
        self.spider_syn_embedding = spider_syn_embedding

class TableMapping:
    """
    Speichert die Zuordnung zwischen dem verarbeiteten Tabellennamen und dem Originalnamen.
    """
    def __init__(self, table_name: str, original_table_name: str):
        self.table_name = table_name
        self.original_table_name = original_table_name

class ColumnMapping:
    """
    Speichert die Zuordnung zwischen dem verarbeiteten Spaltennamen und dem Originalnamen.
    Zusätzlich wird festgehalten, zu welcher Tabelle die Spalte gehört.
    """
    def __init__(self, table_name: str, column_name: str, original_column_name: str):
        self.table_name = table_name
        self.column_name = column_name
        self.original_column_name = original_column_name

class MappingDB:
    """
    Speichert für eine bestimmte Datenbank (über db_id) die
    Mapping-Informationen der Tabellen und Spalten, sowie zugehörige Embeddings.
    """
    def __init__(self, 
                 db_id: str, 
                 table_mappings: List[TableMapping], 
                 column_mappings: List[ColumnMapping]):
        self.db_id = db_id
        self.table_mappings = table_mappings
        self.column_mappings = column_mappings
        # Hier werden Embeddings als Dictionary abgelegt:
        # key: table_name bzw. column_name, value: np.ndarray (Embedding-Vektor)
        self.table_embeddings: Dict[str, np.ndarray] = {}
        self.column_embeddings: Dict[str, np.ndarray] = {}

    def add_table_embedding(self, table_name: str, embedding: np.ndarray):
        self.table_embeddings[table_name] = embedding

    def add_column_embedding(self, column_name: str, embedding: np.ndarray):
        self.column_embeddings[column_name] = embedding

class SpiderDatabase:
    """
    Schnittstelle zu einer gefüllten SQLite-Datenbank,
    die die tatsächlichen Daten aus dem Spider-Datenset enthält.
    """
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.connection = sqlite3.connect(db_path)
    
    def query(self, sql_query: str):
        cursor = self.connection.cursor()
        cursor.execute(sql_query)
        return cursor.fetchall()

    def close(self):
        self.connection.close()


In [106]:
import json
import numpy as np
from typing import List, Dict, Optional

# Annahme: Die Klassen wurden bereits definiert:
# QuestionAnswerMapping, TableMapping, ColumnMapping, MappingDB

# Datei-Pfad zum JSON-Datensatz
# file_path_dev = "Spider-Syn-main/preprocessed_dataset/dev.json"
file_path_qa = "Spider-Syn-main/Spider-Syn/dev.json"


# Lade die JSON-Daten
with open(file_path_qa, "r", encoding="utf-8") as f:
    data = json.load(f)

# Listen und Dictionaries zur Speicherung der erstellten Objekte
question_answer_mappings: List[QuestionAnswerMapping] = []
mapping_dbs: Dict[str, MappingDB] = {}

for item in data:
    # Extrahiere die Basisinformationen aus dem JSON-Eintrag
    db_id = item.get("db_id", "")
    spider_question = item.get("SpiderQuestion", "")
    # Hier nehmen wir an, dass auch die synthetisierte Frage im Feld "question" liegt.
    # Falls es einen separaten Key gäbe, müsste hier angepasst werden.
    spider_syn_question = item.get("SpiderSynQuestion", "")
    query = item.get("query", "")
    # Initiales Embedding auf None setzen – kann später befüllt werden
    spider_syn_embedding = None

    # Erstelle ein QuestionAnswerMapping-Objekt
    qam = QuestionAnswerMapping(
        db_id=db_id,
        spider_question=spider_question,
        spider_syn_question=spider_syn_question,
        query=query,
        spider_syn_embedding=spider_syn_embedding
    )
    question_answer_mappings.append(qam)

print(f"Es wurden {len(question_answer_mappings)} QuestionAnswerMapping-Objekte erstellt.")



Es wurden 1034 QuestionAnswerMapping-Objekte erstellt.


In [103]:
if __name__ == "__main__":
    file_path_qa = "Spider-Syn-main/Spider-Syn/dev.json"
    qa_mappings, mapping_dbs = load_question_answer_mappings(file_path_qa)

Es wurden 1034 QuestionAnswerMapping-Objekte erstellt.


In [105]:
mapping_dbs

{}

In [104]:
qa_mappings

 ...]

In [97]:
import json
from typing import List, Dict
import numpy as np

# Wir nehmen an, dass die Klassen schon definiert sind:
# TableMapping, ColumnMapping, MappingDB

file_path_tables = "Spider-Syn-main/preprocessed_dataset/tables.json"

with open(file_path_tables, "r", encoding="utf-8") as f:
    tables_data = json.load(f)

mapping_dbs: Dict[str, MappingDB] = {}

# Die Variable tables_data ist eine LISTE, in der jeder Eintrag das Schema einer DB beschreibt
for db_schema in tables_data:
    # 1) DB-spezifische Infos auslesen
    db_id = db_schema["db_id"]
    table_names = db_schema["table_names"]                 # ["table1", "table2", ...]
    table_names_original = db_schema["table_names_original"] # ["Table1", "Table2", ...] (oder ähnlich)
    column_names = db_schema["column_names"]               # [[table_idx, "column_name"], ...]
    column_names_original = db_schema["column_names_original"] # [[table_idx, "column_name_orig"], ...]

    # 2) TableMapping-Objekte erstellen
    table_mappings: List[TableMapping] = []
    for tname, tname_orig in zip(table_names, table_names_original):
        table_mappings.append(TableMapping(
            table_name=tname,
            original_table_name=tname_orig
        ))

    # 3) ColumnMapping-Objekte erstellen
    column_mappings: List[ColumnMapping] = []
    for (tbl_idx, col_name), (_, col_name_orig) in zip(column_names, column_names_original):
        if tbl_idx == -1:
            # -1 bedeutet "Sternchen" oder keine bestimmte Tabelle
            # Wir können das Tabellennamen-Feld leer lassen oder "*"
            table_name = ""
        else:
            # Ansonsten den Tabellenindex benutzen, um den zugehörigen Tabellennamen zu holen
            # und sicherstellen, dass wir nicht out-of-bounds laufen
            try:
                table_name = table_names[tbl_idx]
            except IndexError:
                table_name = ""
        
        column_mappings.append(ColumnMapping(
            table_name=table_name,
            column_name=col_name,
            original_column_name=col_name_orig
        ))
    
    # 4) MappingDB-Objekt für diese Datenbank erstellen
    mapping_dbs[db_id] = MappingDB(
        db_id=db_id,
        table_mappings=table_mappings,
        column_mappings=column_mappings
    )

print(f"Erstellte MappingDB-Objekte: {len(mapping_dbs)}")


Erstellte MappingDB-Objekte: 166


In [98]:
import json

# Wähle ein Beispielobjekt aus
example_db = next(iter(mapping_dbs.values()))

# Benutzerdefinierte Funktion für den JSON-Encoder
def custom_encoder(obj):
    if hasattr(obj, '__dict__'):
        return obj.__dict__
    # Bei Listen oder anderen iterierbaren Typen wird rekursiv serialisiert
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

# Serialisiere das Objekt unter Verwendung der custom_encoder-Funktion
json_str = json.dumps(example_db, default=custom_encoder, indent=2, ensure_ascii=False)
print(json_str)


{
  "db_id": "perpetrator",
  "table_mappings": [
    {
      "table_name": "perpetrator",
      "original_table_name": "perpetrator"
    },
    {
      "table_name": "people",
      "original_table_name": "people"
    }
  ],
  "column_mappings": [
    {
      "table_name": "",
      "column_name": "*",
      "original_column_name": "*"
    },
    {
      "table_name": "perpetrator",
      "column_name": "perpetrator id",
      "original_column_name": "Perpetrator_ID"
    },
    {
      "table_name": "perpetrator",
      "column_name": "people id",
      "original_column_name": "People_ID"
    },
    {
      "table_name": "perpetrator",
      "column_name": "date",
      "original_column_name": "Date"
    },
    {
      "table_name": "perpetrator",
      "column_name": "year",
      "original_column_name": "Year"
    },
    {
      "table_name": "perpetrator",
      "column_name": "location",
      "original_column_name": "Location"
    },
    {
      "table_name": "perpetrator",
     

In [99]:
print(f"Es existieren folgende Tabellen:\n{list(mapping_dbs.keys())}\n\n")

print(f"Anzahl Tabellen insgesamt:\n{len(list(mapping_dbs.keys()))}")

Es existieren folgende Tabellen:
['perpetrator', 'college_2', 'flight_company', 'icfp_1', 'body_builder', 'storm_record', 'pilot_record', 'race_track', 'academic', 'department_store', 'music_4', 'insurance_fnol', 'cinema', 'decoration_competition', 'phone_market', 'store_product', 'assets_maintenance', 'student_assessment', 'dog_kennels', 'music_1', 'company_employee', 'farm', 'solvency_ii', 'city_record', 'swimming', 'flight_2', 'election', 'manufactory_1', 'debate', 'network_2', 'local_govt_in_alabama', 'climbing', 'e_learning', 'scientist_1', 'ship_1', 'entertainment_awards', 'allergy_1', 'imdb', 'products_for_hire', 'candidate_poll', 'chinook_1', 'flight_4', 'pets_1', 'dorm_1', 'journal_committee', 'flight_1', 'medicine_enzyme_interaction', 'local_govt_and_lot', 'station_weather', 'shop_membership', 'driving_school', 'concert_singer', 'music_2', 'sports_competition', 'railway', 'inn_1', 'museum_visit', 'browser_web', 'baseball_1', 'architecture', 'csu_1', 'tracking_orders', 'insura

In [101]:
# Beispiel: Erstes QuestionAnswerMapping-Objekt anzeigen
first_qam = question_answer_mappings[912]

print("=== QuestionAnswerMapping ===")
print(f"DB ID: {first_qam.db_id}")
print(f"Originalfrage (Spider): {first_qam.spider_question}")
print(f"Synthetisierte Frage (SpiderSyn): {first_qam.spider_syn_question}")
print(f"Query: {first_qam.query}")
print(f"Generated Query: {first_qam.generated_query}")
print(f"Embedding (SpiderSyn): {first_qam.spider_syn_embedding}")


=== QuestionAnswerMapping ===
DB ID: network_1
Originalfrage (Spider): How many likes does Kyle have?
Synthetisierte Frage (SpiderSyn): How many interests does Kyle have?
Query: SELECT count(*) FROM Likes AS T1 JOIN Highschooler AS T2 ON T1.student_id  =  T2.id WHERE T2.name  =  "Kyle"
Generated Query: None
Embedding (SpiderSyn): None


In [79]:
print(vars(first_qam))

{'db_id': 'concert_singer', 'spider_question': 'What is the average , minimum , and maximum age for all France singers ?', 'spider_syn_question': 'What is the average , minimum , and maximum age for all France singers ?', 'query': "SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'", 'generated_query': None, 'spider_syn_embedding': None}


In [72]:
# Hole das erste MappingDB-Objekt aus dem Dictionary
first_db_id = list(mapping_dbs.keys())[19]
first_mapping_db = mapping_dbs[first_db_id]

print("=== MappingDB ===")
print(f"DB ID: {first_mapping_db.db_id}")

print("\nTabellen-Mappings:")
for table in first_mapping_db.table_mappings:
    print(f"- {table.table_name} (Original: {table.original_table_name})")

print("\nSpalten-Mappings:")
for column in first_mapping_db.column_mappings:
    print(f"- {column.column_name} (Original: {column.original_column_name}, Tabelle: {column.table_name})")

print("\nTable-Embeddings (nur Keys):")
print(list(first_mapping_db.table_embeddings.keys()))

print("\nColumn-Embeddings (nur Keys):")
print(list(first_mapping_db.column_embeddings.keys()))


=== MappingDB ===
DB ID: music_1

Tabellen-Mappings:
- genre (Original: genre)
- artist (Original: artist)
- files (Original: files)
- song (Original: song)

Spalten-Mappings:
- * (Original: *, Tabelle: )
- genre name (Original: g_name, Tabelle: genre)
- rating (Original: rating, Tabelle: genre)
- most popular in (Original: most_popular_in, Tabelle: genre)
- artist name (Original: artist_name, Tabelle: artist)
- country (Original: country, Tabelle: artist)
- gender (Original: gender, Tabelle: artist)
- preferred genre (Original: preferred_genre, Tabelle: artist)
- song id (Original: f_id, Tabelle: files)
- artist name (Original: artist_name, Tabelle: files)
- file size (Original: file_size, Tabelle: files)
- duration (Original: duration, Tabelle: files)
- formats (Original: formats, Tabelle: files)
- song name (Original: song_name, Tabelle: song)
- artist name (Original: artist_name, Tabelle: song)
- country (Original: country, Tabelle: song)
- song id (Original: f_id, Tabelle: song)
-